# Выбор локации для скважины

## Задача

Необходимо определить локацию для бурения новой скважины.

Шаги выбора локации:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Имеется информация о пробах нефти в трех регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построить модель для определения региона, где добыча принесет наибольшую прибыль. Проанализировать прибыль и риски техникой Bootstrap

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# Шаг 1. Подготовка данных

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except FileNotFoundError:
    data_0 = pd.read_csv('geo_data_0.csv')
    data_1 = pd.read_csv('geo_data_1.csv')
    data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
# функция открытия набора данных
def open_file(df):
    display(df.head())
    display(df.info())

In [4]:
open_file(data_0)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
open_file(data_1)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [6]:
open_file(data_2)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Описание данных от заказчика
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

<b>Предварительный вывод</b>
- данные загружены
- пропусков в таблицах нет
- данные соответствуют описанию

<b>Выводы</b>
- данные загружены
- пропусков нет
- проанализированы все три локации:
    - большая часть данных распределена нормально
    - в некоторые столбцах имеются выбросы, но они не влияют на распределение данных
    - неизвестные данные в столбцах f0, f1, f2 имеют сопоставимые порядки величин

# Шаг 2. Обучение модели лин.рег. и проверка ее на каждом регионе

Для начала разделим данные на признаки и целевой

In [7]:
features_geo_0 = data_0.drop(['product', 'id'], axis=1)
target_geo_0 = data_0['product']

features_geo_1 = data_1.drop(['product', 'id'], axis=1)
target_geo_1 = data_1['product']

features_geo_2 = data_2.drop(['product', 'id'], axis=1)
target_geo_2 = data_2['product']

In [8]:
def model_fit(features, target): 
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, \
                                                                            test_size=0.25, random_state=12345)
    print('-----------------------------------------------------')
    print('Общий размер выборки:', features.shape)
    print('Размер тренировочной выборки:', features_train.shape)
    print('Размер валидационной выборки:', features_valid.shape)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid_mean = predicted_valid.mean()
    rmse = mean_squared_error(target_valid, predicted_valid) ** .5
    
    print('Среднее кол-во сырья в локации:', round(predicted_valid_mean, 3))
    print('RMSE:', round(rmse, 3))
    print('-----------------------------------------------------')
    print()
    
    return target_valid, predicted_valid

In [9]:
# локация 0
target_geo_0_valid, predicted_geo_0_valid = model_fit(features_geo_0, target_geo_0)

# локация 1
target_geo_1_valid, predicted_geo_1_valid = model_fit(features_geo_1, target_geo_1)

# локация 2
target_geo_2_valid, predicted_geo_2_valid = model_fit(features_geo_2, target_geo_2)

-----------------------------------------------------
Общий размер выборки: (100000, 3)
Размер тренировочной выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)
Среднее кол-во сырья в локации: 92.593
RMSE: 37.579
-----------------------------------------------------

-----------------------------------------------------
Общий размер выборки: (100000, 3)
Размер тренировочной выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)
Среднее кол-во сырья в локации: 68.729
RMSE: 0.893
-----------------------------------------------------

-----------------------------------------------------
Общий размер выборки: (100000, 3)
Размер тренировочной выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)
Среднее кол-во сырья в локации: 94.965
RMSE: 40.03
-----------------------------------------------------



<b>Вывод</b>
- в локациях 0 и 2 на 30% больше сырья чем в локации 1
- RMSE в локация 0 и 2 на порядки больше чем в локации 1, что свидетельствует о том, что сырья в локации 1 в среднем предсказано меньше, но данный прогноз в разы точнее чем в локациях 0 и 1

# Шаг 3. Подготовка к расчету прибыли

- объем средств для освоения - 10 млрд
- кол-во скважин - 200
- доход с одной единицы сырья - 450 тыс

In [10]:
BUDGET = 10 ** 10
TOP_WELL = 200
WELL_INCOME = 450 * 10 ** 3

In [11]:
min_brent_amount = BUDGET / TOP_WELL / WELL_INCOME
print('Минимальный объем нефти с одной скважины:', round(min_brent_amount, 2))

Минимальный объем нефти с одной скважины: 111.11


# Шаг 4. Расчет прибыли

Напишем функции для расчета прибыли и убытков

In [12]:
target_geo_0_valid = target_geo_0_valid.reset_index(drop=True)
target_geo_1_valid = target_geo_1_valid.reset_index(drop=True)
target_geo_2_valid = target_geo_2_valid.reset_index(drop=True)

predicted_geo_0_valid = pd.Series(predicted_geo_0_valid)
predicted_geo_1_valid = pd.Series(predicted_geo_1_valid)
predicted_geo_2_valid = pd.Series(predicted_geo_2_valid)

Напишем функцию для расчета прибыли по выбранным скважинам и предсказаниями модели для получения целевого объема сырья исходя из предсказаний и прибыли

In [13]:
state = np.random.RandomState(12345)

def predicted_income(target, predictions, name):
    sample_preds = predictions.sample(n=500, replace=True, random_state=state)
    top_preds = sample_preds.sort_values(ascending=False)[:TOP_WELL]
    top_targets = target[top_preds.index]
    top_wells_mean = top_targets.mean()
    print('-----------------------------------------------------')
    print('Локация: ' + name)
    print('Средний запас сырья среди скважин с максимальным показателем: {:.2f}'.format(top_wells_mean))
    print('-----------------------------------------------------')
    print()

In [14]:
for target, prediction, name in [(target_geo_0_valid, predicted_geo_0_valid, 'geo_0'), 
                                 (target_geo_1_valid, predicted_geo_1_valid, 'geo_1'), 
                                 (target_geo_2_valid, predicted_geo_2_valid, 'geo_2')]:
    predicted_income(target, prediction, name)

-----------------------------------------------------
Локация: geo_0
Средний запас сырья среди скважин с максимальным показателем: 117.84
-----------------------------------------------------

-----------------------------------------------------
Локация: geo_1
Средний запас сырья среди скважин с максимальным показателем: 114.83
-----------------------------------------------------

-----------------------------------------------------
Локация: geo_2
Средний запас сырья среди скважин с максимальным показателем: 118.07
-----------------------------------------------------



<b>Вывод</b>
- средний запас сырья в скважинах с максимальным показателем превосходит минимально необходимый на всех локациях

# Шаг 5. Расчет прибыли и рисков для каждой локации

In [15]:
def profit(target, predictions):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:TOP_WELL]
    return (selected.sum() * WELL_INCOME - BUDGET) / 1_000_000

In [16]:
def result(target,predictions, name):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        result = profit(target_subsample, probs_subsample)
        values.append(result)
    values = pd.Series(values)
#     percentage_of_negative = (values[values < 0].count() / len(values)) * 100
    percentage_of_negative = (values < 0).mean() * 100
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('-----------------------------------------------------')
    print('Локация: ' + name)
    print(f'Средняя прогнозная прибыль равняется {mean:.2f} млн')
    print(f'С вероятностью 95% прибыль составит от {lower:.2f} до {upper:.2f} млн')
    print(f'Риск убытков составляет {percentage_of_negative}%')
    print('-----------------------------------------------------')
    print()

In [17]:
for target, prediction, name in [(target_geo_0_valid, predicted_geo_0_valid, 'geo_0'), 
                                 (target_geo_1_valid, predicted_geo_1_valid, 'geo_1'), 
                                 (target_geo_2_valid, predicted_geo_2_valid, 'geo_2')]:
    result(target, prediction, name)

-----------------------------------------------------
Локация: geo_0
Средняя прогнозная прибыль равняется 425.18 млн
С вероятностью 95% прибыль составит от -102.09 до 947.98 млн
Риск убытков составляет 6.0%
-----------------------------------------------------

-----------------------------------------------------
Локация: geo_1
Средняя прогнозная прибыль равняется 519.38 млн
С вероятностью 95% прибыль составит от 131.58 до 953.61 млн
Риск убытков составляет 0.3%
-----------------------------------------------------

-----------------------------------------------------
Локация: geo_2
Средняя прогнозная прибыль равняется 419.92 млн
С вероятностью 95% прибыль составит от -115.85 до 989.63 млн
Риск убытков составляет 6.2%
-----------------------------------------------------



<b>Выводы</b>
- Самая высокая оценка средней прибыли в локации 1 - 503.44 млн.
- Только в локации 1 разработка безубыточная в 95% доверительном интервале
- риск убытков для для локации 1 0.9%

# Шаг 6. Выводы

1. Загрузка данных
    - данные загружены
    - пропусков нет
    - наблюдаются небольшие выбросы
2. Начальное построение модели:
    - в локациях 0 и 2 на 30% больше сырья чем в локации 1
    - RMSE в локация 0 и 2 на порядки больше чем в локации 1, что свидетельствует о том, что сырья в локации 1 в среднем предсказано меньше, но данный прогноз в разы точнее чем в локациях 0 и 1
    - средние показатели сырья ниже минимально необходимого
3. Необходимый минимальный запас сырья при имеющимся бюджете - 111,11
4. Запас сырья в 200 наилучших скважинах в среднем превосходит минимально необходимый во всех локациях
5. Оценка прибылей и убытков
    - Самая высокая оценка средней прибыли в локации 1 - 503.44 млн.
    - Только в локации 1 разработка безубыточная в 95% доверительном интервале
    - риск убытков для для локации 1 0.9%
    
В итоге рекомендована разработка в локации 1